#Setup

In [ ]:
# seleziona directory di lavoro
%cd D:\opimi_test\ 

# Mapping dataset review
## Mapping id dei film in _entities.txt_ con l'id di IMDB
Partendo da una cartella contenente i file _entities.txt_, _entityPropertyMapping.txt_ e tutti i file del dataset con le recensioni, chiamati  _ttxxxxxxx_ (ossia con l'id IMDB del film che trattano) generiamo il file _idMapping.txt_ contenente nella prima colonna l'id del film (da _entities.txt_) e nella seconda colonna l'id di IMDB (prelevato da _entityPropertyMapping_).

In [ ]:
import pandas as pd

entities = pd.read_csv("entities.txt", sep="|", header=None, names=["movie_id", "movie_name"])
entities = entities.drop_duplicates(subset="movie_id")

entityPropertyMapping = pd.read_csv("entityPropertyMapping.txt", sep="|", header=None, names=["movie_id", "property_type", "property"])
imdbMapping = entityPropertyMapping.drop(entityPropertyMapping[entityPropertyMapping.property_type != "P345"].index) #mantieni solo le righe contenenti id IMDB identificati da "P345" nella seconda colonna
imdbMapping = imdbMapping.drop_duplicates(subset="movie_id")

idMapping = entities.merge(imdbMapping, on="movie_id")
idMapping.to_csv("idMapping.txt", sep="|", header=None, index=None, columns=["movie_id", "property"])
idMapping = pd.read_csv("idMapping.txt", sep="|", header=None, names=["movie_id", "imdb_id"])

idMapping.head()

## Mapping dei film in _idMapping.txt_ con le review
Utilizzando il file _idMapping.txt_ appena generato mappiamo le recensioni con i file _ttxxxxxxx_ che contengono le recensioni in formato JSON valutando contestualmente quanti film abbiamo coperto.

In [ ]:
import json
import os

total = unmapped = 0

try:
    idMapping = open("idMapping.txt", "r", encoding="UTF8")
    entityReviewMapping = open("entityReviewMapping.txt", "w", encoding="UTF8")

    for line_id in idMapping:
        total += 1
        splitted_line = line_id.split("|")
        movie_id = splitted_line[0]
        imdb_id = splitted_line[1][:-1]

        if os.path.exists(imdb_id) and os.stat(imdb_id).st_size != 0:
            reviews_file = open(imdb_id, "r", encoding="UTF8")

            for line_re in reviews_file:
                review = json.loads(line_re)
                review_id = review["id"]
                review_text = review["text"]
                entityReviewMapping.write(movie_id + "|" + review_id + "|" + review_text + "\n")
            reviews_file.close()
        else:
            unmapped +=1
finally:
    idMapping.close()
    entityReviewMapping.close()

entityReviewMapping = pd.read_csv("entityReviewMapping.txt", sep="|", header=None, names=["movie_id", "review_id", "review_text"])
entityReviewMapping.head()

In [ ]:
print("Number of films: " + str(total) + "\n")
print("Number of mapped films: " + str(total-unmapped) + "\n")
print("Number of unmapped films: " + str(unmapped) + "\n")

## Numero di recensioni per film, minimo, massimo e media
Utilizzando il file _entityReviewMapping.txt_ calcoliamo il numero di recensioni per ogni film, che verrà salvato in _reviewsPerFilm.txt_, e il numero minimo, massimo e medio di recensioni totali.

In [ ]:
entityReviewMapping["movie_id"].value_counts().reset_index().to_csv("reviewsPerFilm.txt", sep="|", index=False, header=False)
reviewsPerFilm = pd.read_csv("reviewsPerFilm.txt", sep="|", header=None, names=["movie_id", "n_reviews"])

reviewsPerFilm.head()

In [ ]:
print("Minimum number of reviews: " + str(reviewsPerFilm.n_reviews.min()))
print("Maximum number of reviews: " + str(reviewsPerFilm.n_reviews.max()))
print("Mean number of reviews: " + str(reviewsPerFilm.n_reviews.mean()))

## Lunghezza delle recensioni, minimo, massimo e media
Utilizzando il file _entityReviewMapping.txt_ calcoliamo la lunghezza di ogni recensione che verrà salvata insieme ai relativi ID in _reviewsLength.txt_, quindi calcoliamo lunghezza minima, massima e media delle recensioni per ogni film e la salviamo in _reviewsStatistics.txt_.

In [ ]:
import numpy as np
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

entityReviewMapping["review_length"] = entityReviewMapping.apply(lambda row: len(tokenizer.tokenize(str(row["review_text"]))), axis=1)
entityReviewMapping.to_csv("reviewsLength.txt", sep="|", index=False, header=False, columns=["movie_id", "review_id", "review_length"])
reviewsLength = pd.read_csv("reviewsLength.txt", sep="|", header=None, names=["movie_id", "review_id", "review_length"])

reviewsLength["min_length"] = reviewsLength.groupby(["movie_id"], as_index=False)["review_length"].transform(min)
reviewsLength["max_length"] = reviewsLength.groupby(["movie_id"], as_index=False)["review_length"].transform(max)
reviewsLength["mean_length"] = reviewsLength.groupby(["movie_id"], as_index=False)["review_length"].transform(np.mean)

reviewsStatistics = reviewsLength.drop_duplicates(subset="movie_id")

del reviewsLength["min_length"]
del reviewsLength["max_length"]
del reviewsLength["mean_length"]
del reviewsStatistics["review_length"]
del reviewsStatistics["review_id"]

reviewsStatistics.to_csv("reviewsStatistics.txt", sep="|", index=False, header=False, columns=["movie_id", "min_length", "max_length", "mean_length"])

reviewsLength.head()

In [ ]:
reviewsStatistics.head()

# Creazione lista di ID dei film con recensioni

In [ ]:
entityReviewMapping = pd.read_csv("entityReviewMapping.txt", sep="|", header=None, names=["movie_id", "review_id", "review_text"])
del entityReviewMapping['review_id']
del entityReviewMapping['review_text']
entityReviewMapping.drop_duplicates(inplace = True)
entityReviewMapping.to_csv('dataset_movie_list.txt', index=None, header=None)